In [21]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing as prepro
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
from matplotlib import pyplot as plt


In [22]:
balt_crime_trends =pd.read_csv("resources/BaltimoreCrimeData.csv/BaltimoreCrimeData.csv")

C:\Users\mitch\anaconda3\1\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
balt_crime_trends.sample(5)

,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
480464,480465,12K04429,2012/11/10 23:09:00+00,3B,1 HOPKINS PZ,ROBBERY,O,NaN,114.0,M,32.0,BLACK_OR_AFRICAN_AMERICAN,NaN,CENTRAL,DOWNTOWN,39.289268,-76.617710,"(39.289267734884938,-76.617709595127053)",BUS/AUTO,1
489147,489148,12G07602,2012/07/13 19:00:00+00,5A,500 N GLOVER ST,BURGLARY,I,NaN,221.0,F,30.0,BLACK_OR_AFRICAN_AMERICAN,NaN,SOUTHEAST,MCELDERRY PARK,39.297607,-76.580157,"(39.297607,-76.580157)",ROW/TOWNHOUSE-OCC,1
227928,227929,17L02397,2017/12/06 19:30:00+00,3AJF,5300 GOODNOW RD,ROBBERY - CARJACKING,O,FIREARM,443.0,F,NaN,BLACK_OR_AFRICAN_AMERICAN,NaN,NORTHEAST,FRANKFORD,39.323634,-76.543045,"(39.323634,-76.543045)",PARKING LOT-OUTSIDE,1
292392,292393,16J02508,2016/10/06 18:35:00+00,4E,100 N PACA ST,COMMON ASSAULT,I,NaN,114.0,M,19.0,BLACK_OR_AFRICAN_AMERICAN,NaN,CENTRAL,DOWNTOWN,39.290888,-76.622458,"(39.290887887178904,-76.622457736079753)",APT/CONDO - OCCUPIED,1
532188,532189,11I07239,2011/09/15 21:00:00+00,6D,2600 BOSTON ST,LARCENY FROM AUTO,O,NaN,214.0,F,31.0,WHITE,NaN,SOUTHEAST,CANTON,39.280267,-76.579621,"(39.280267,-76.579621)",STREET,1


In [28]:
balt_crime_trends.dropna()

,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
0,1,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,nan,PERSONAL_WEAPONS,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,(39.27330200992213-76.69243902745305),nan,1
1,2,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,nan,PERSONAL_WEAPONS,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,(39.27330200992213-76.69243902745305),nan,1
2,3,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,nan,PERSONAL_WEAPONS,833.0,F,27.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,(39.27330200992213-76.69243902745305),nan,1
3,4,23F08231,2023/06/24 04:01:00+00,3JK,600 LUCIA AVE,ROBBERY,nan,PERSONAL_WEAPONS,833.0,M,25.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,(39.27330200992213-76.69243902745305),nan,1
4,5,23F08235,2023/06/24 03:45:00+00,5A,3200 LILY AVE,BURGLARY,nan,nan,922.0,M,48.0,nan,HISPANIC_OR_LATINO,SOUTHERN,CHERRY HILL,39.246432101114614,-76.63681903810716,(39.24643210111462-76.63681903810716),nan,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568609,568610,11C00465,2011/03/01 22:11:00+00,4D,600 MELVILLE AVE,AGG. ASSAULT,I,HANDS,515.0,F,25.0,BLACK_OR_AFRICAN_AMERICAN,nan,NORTHERN,WAVERLY,39.331681,-76.607352,(39.331681-76.607352),ROW/TOWNHOUSE-OCC,1
568610,568611,11C00686,2011/03/01 20:30:00+00,6J,2700 ANNAPOLIS RD,LARCENY,I,nan,921.0,nan,nan,UNKNOWN,nan,SOUTHERN,WESTPORT,39.25627,-76.638018,(39.25627-76.638018),OTHER - INSIDE,1
568611,568612,11C00912,2011/03/01 20:00:00+00,7A,900 LYNDHURST ST,AUTO THEFT,O,nan,815.0,M,38.0,BLACK_OR_AFRICAN_AMERICAN,nan,SOUTHWEST,EDMONDSON VILLAGE,39.297945,-76.681485,(39.297945-76.681485),STREET,1
568612,568613,11C00337,2011/03/01 14:40:00+00,4B,1600 LORMAN CT,AGG. ASSAULT,O,KNIFE,724.0,M,42.0,BLACK_OR_AFRICAN_AMERICAN,nan,WESTERN,SANDTOWN-WINCHESTER,39.304108,-76.64387099999999,(39.304108-76.643871),OTHER - OUTSIDE,1


In [29]:
balt_crime_trends = balt_crime_trends.astype(str)
balt_crime_trends = balt_crime_trends.applymap(lambda x: x.replace(',', ''))


In [30]:
balt_crime_trends.head(10)

,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
0,1,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,nan,PERSONAL_WEAPONS,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,(39.27330200992213-76.69243902745305),nan,1
1,2,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,nan,PERSONAL_WEAPONS,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,(39.27330200992213-76.69243902745305),nan,1
2,3,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,nan,PERSONAL_WEAPONS,833.0,F,27.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,(39.27330200992213-76.69243902745305),nan,1
3,4,23F08231,2023/06/24 04:01:00+00,3JK,600 LUCIA AVE,ROBBERY,nan,PERSONAL_WEAPONS,833.0,M,25.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,(39.27330200992213-76.69243902745305),nan,1
4,5,23F08235,2023/06/24 03:45:00+00,5A,3200 LILY AVE,BURGLARY,nan,nan,922.0,M,48.0,nan,HISPANIC_OR_LATINO,SOUTHERN,CHERRY HILL,39.246432101114614,-76.63681903810716,(39.24643210111462-76.63681903810716),nan,1
5,6,23F08342,2023/06/24 12:35:00+00,4E,1800 SPENCE ST,COMMON ASSAULT,nan,PERSONAL_WEAPONS,831.0,M,30.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHWEST,MORRELL PARK,39.26852000843198,-76.65114298306925,(39.26852000843198-76.65114298306925),nan,1
6,7,23F08317,2023/06/24 12:00:00+00,4E,1200 CLEVELAND ST,COMMON ASSAULT,nan,PERSONAL_WEAPONS,932.0,F,41.0,WHITE,NOT_HISPANIC_OR_LATINO,SOUTHERN,WASHINGTON VILLAGE/PIGTOWN,39.280127980411116,-76.63258201285849,(39.28012798041112-76.6325820128585),nan,1
7,8,23F08321,2023/06/24 12:20:00+00,4E,900 HERNDON CT,COMMON ASSAULT,nan,PERSONAL_WEAPONS,913.0,F,21.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHERN,BROOKLYN,39.22830342820063,-76.60081926609222,(39.228303428200626-76.60081926609222),nan,1
8,9,23F08198,2023/06/24 01:50:00+00,4E,400 N HAVEN ST,COMMON ASSAULT,nan,PERSONAL_WEAPONS,223.0,M,40.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHEAST,BALTIMORE HIGHLANDS,39.29648798857653,-76.56325998556021,(39.29648798857653-76.56325998556021),nan,1
9,10,23F08199,2023/06/24 01:30:00+00,3AF,4800 EDMONDSON AVE,ROBBERY,nan,HANDGUN,822.0,M,29.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,HUNTING RIDGE,39.292962015277226,-76.70089796031867,(39.292962015277226-76.70089796031867),nan,1


In [31]:
balt_crime=balt_crime_trends.drop(['Inside_Outside','Weapon','GeoLocation','Premise'], axis=1)
balt_crime.head(10)

,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,Total_Incidents
0,1,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,1
1,2,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,1
2,3,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,833.0,F,27.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,1
3,4,23F08231,2023/06/24 04:01:00+00,3JK,600 LUCIA AVE,ROBBERY,833.0,M,25.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHWEST,YALE HEIGHTS,39.27330200992213,-76.69243902745305,1
4,5,23F08235,2023/06/24 03:45:00+00,5A,3200 LILY AVE,BURGLARY,922.0,M,48.0,nan,HISPANIC_OR_LATINO,SOUTHERN,CHERRY HILL,39.246432101114614,-76.63681903810716,1
5,6,23F08342,2023/06/24 12:35:00+00,4E,1800 SPENCE ST,COMMON ASSAULT,831.0,M,30.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHWEST,MORRELL PARK,39.26852000843198,-76.65114298306925,1
6,7,23F08317,2023/06/24 12:00:00+00,4E,1200 CLEVELAND ST,COMMON ASSAULT,932.0,F,41.0,WHITE,NOT_HISPANIC_OR_LATINO,SOUTHERN,WASHINGTON VILLAGE/PIGTOWN,39.280127980411116,-76.63258201285849,1
7,8,23F08321,2023/06/24 12:20:00+00,4E,900 HERNDON CT,COMMON ASSAULT,913.0,F,21.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHERN,BROOKLYN,39.22830342820063,-76.60081926609222,1
8,9,23F08198,2023/06/24 01:50:00+00,4E,400 N HAVEN ST,COMMON ASSAULT,223.0,M,40.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHEAST,BALTIMORE HIGHLANDS,39.29648798857653,-76.56325998556021,1
9,10,23F08199,2023/06/24 01:30:00+00,3AF,4800 EDMONDSON AVE,ROBBERY,822.0,M,29.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,HUNTING RIDGE,39.292962015277226,-76.70089796031867,1


In [32]:
balt_crime.to_csv("resources/balt_cleaned.csv", index=False)